# 01 Data Preparation

The dataset chosen for clustering using the SOM technique is the 'Online Shopper Intention' dataset (Sakar, Polat, S.O., Katircioglu, M. et al., 2018), available from the [UCI Machine Learning repository](http://archive.ics.uci.edu/ml/datasets/Online+Shoppers+Purchasing+Intention+Dataset) (Dua and Graff, 2019).

In this notebook the data is loaded from CSV to Pandas, and transformed to apply featurisation and normalisation. Finally the dataset is split into training, validation and testing sets and serialised for use in subsequent notebooks.

## 1.1 Dataset

The 'Online Shopper Intention' dataset contains information from more than 12000 browsing sessions on a website with an online store. Each observation includes the following information:

#### Web pages

The features 'Administrative', 'Informational', and 'ProductRelated' refer to the number of pages of given class that were visited by the user. For each type of webpage there is also a '_ Duration' feature referring to the total time spent on each type of page.

### Session information

Each observation includes the 'Bounce Rate', 'Exit Rate', and 'Page Value' of the pages visited, as calculated by Google Analytices (). 

- The bounce rate of a page is the percentage of visitors who enter the site from the page and leave without taking further actions on the website
- The exit rate of a page is the percentage of views of the page that are the last in a given browsing session
- The page value of a page is calculated by averaging the money spent by a visitor who completes a transaction immediately after visiting the page

Each observation also includes a boolean value 'revenue' indicating whether or not the visitor made a purchase. 84.5% of the users did not make a purchase.

### Date information

Each observation includes:

- the month of the year that the browsing session took place 
- whether or not the browsing session took place close to a 'Special Day' (e.g. between February 8th and February 12th - a period of high likelihood of shopping for Valentine's Day)

### User information
operating system, browser, region, traffic type, visitor type as returning or new visitor
Each observation contains information about the user's

- operating system
- browswer
- region
- traffic type
- visitor type (either 'returning' or 'new')

## 1.2 Justification for use of dataset

Almost all e-commerce companies monitor web traffic, but few are able to dissect the enormous amount of data provided by Google Analytics (Google) and use it to understand customer behaviour and drive sales (Matous, 2015). With more than 29 000 000 websites making use of Google Analytics (builtwith.com, 2020), and each browsing session generating a large number of features including those included with this dataset, in order for small businesses to gain competitive advantage it is vital to have the capability to make use of the data efficiently.

Examining 18 features to understand user behaviour on a website is an arduous task, and for a large e-commerce organisation that can generate millions of observations in a day it can become costly to store and process the data. By training a self-organising map (SOM) on this dataset, the features can be projected onto a 2D space for inspection. If clusters are identified in the dataset, it implies that certain features correlate with each other, and therefore $n$ correlating features can be reduced to $1$. This reduces the processing burden by allowing a smaller number of features to be used for predictive analytics. For small businesses who operate across multiple retail platforms, this can be key to gaining vital competitive edge (Fitzpatrick, 2019). 

Furthermore, the end-goal of any e-commerce company is to generate revenue. If among the clusters of the trained SOM there is a cluster that correlates with revenue (which in this dataset is a boolean value indicating whether or not a browsing session included a purchase) this enables a company to identify user behaviour and information that is likely to result in a purchase and divert resources appropriately.

Depending on which features are found to relate to 'revenue', businesses could perform such actions as deploying a bot or customer service agent to directly contact browsers who are viewed as likely to purchase, to send marketing emails at times when visitors are likely to purchase, or deploy marketing campaigns that engage visitors on platforms that are likely to convert (for example, mobile apps, if mobile purchases are more likely to generate revenue).  The SOM will not indicate which of these methods is most applicable but it would give marketers insight into which behaviours tend to match their potential customers.


## 1.3 Importing data  

The data is loaded from a CSV into a Pandas DataFrame.


In [1]:
import pandas as pd

input_data_path = 'data/online_shoppers_intention.csv'

data_raw = pd.read_csv(input_data_path)
data_raw.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,1,1,1,1,Returning_Visitor,False,False
1,0,0.0,0,0.0,2,64.000000,0.00,0.10,0.0,0.0,Feb,2,2,1,2,Returning_Visitor,False,False
2,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,4,1,9,3,Returning_Visitor,False,False
3,0,0.0,0,0.0,2,2.666667,0.05,0.14,0.0,0.0,Feb,3,2,2,4,Returning_Visitor,False,False
4,0,0.0,0,0.0,10,627.500000,0.02,0.05,0.0,0.0,Feb,3,3,1,4,Returning_Visitor,True,False


### 1.4.1 Inspecting the data.

In [2]:
data_raw.describe()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,OperatingSystems,Browser,Region,TrafficType
count,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000
mean,2.315166,80.818611,0.503569,34.472398,31.731468,1194.746220,0.022191,0.043073,5.889258,0.061427,2.124006,2.357097,3.147364,4.069586
std,3.321784,176.779107,1.270156,140.749294,44.475503,1913.669288,0.048488,0.048597,18.568437,0.198917,0.911325,1.717277,2.401591,4.025169
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000
25%,0.000000,0.000000,0.000000,0.000000,7.000000,184.137500,0.000000,0.014286,0.000000,0.000000,2.000000,2.000000,1.000000,2.000000
50%,1.000000,7.500000,0.000000,0.000000,18.000000,598.936905,0.003112,0.025156,0.000000,0.000000,2.000000,2.000000,3.000000,2.000000
75%,4.000000,93.256250,0.000000,0.000000,38.000000,1464.157213,0.016813,0.050000,0.000000,0.000000,3.000000,2.000000,4.000000,4.000000
max,27.000000,3398.750000,24.000000,2549.375000,705.000000,63973.522230,0.200000,0.200000,361.763742,1.000000,8.000000,13.000000,9.000000,20.000000


In [3]:
data_raw.dtypes

Administrative               int64
Administrative_Duration    float64
Informational                int64
Informational_Duration     float64
ProductRelated               int64
ProductRelated_Duration    float64
BounceRates                float64
ExitRates                  float64
PageValues                 float64
SpecialDay                 float64
Month                       object
OperatingSystems             int64
Browser                      int64
Region                       int64
TrafficType                  int64
VisitorType                 object
Weekend                       bool
Revenue                       bool
dtype: object

## 1.4 Featurisation

Convert strings to numerical data.

### 1.4.1 Months

In [4]:
data_raw.Month.unique()

array(['Feb', 'Mar', 'May', 'Oct', 'June', 'Jul', 'Aug', 'Nov', 'Sep',
       'Dec'], dtype=object)

In [5]:
months = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 
          'May': 5, 'June': 6, 'Jul': 7, 'Aug': 8, 
          'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}

data_raw['Month'] = data_raw['Month'].replace(months)
data_raw.Month.unique()

array([ 2,  3,  5, 10,  6,  7,  8, 11,  9, 12], dtype=int64)

### 1.4.2 Visitor type

In [6]:
data_raw.VisitorType.unique()

array(['Returning_Visitor', 'New_Visitor', 'Other'], dtype=object)

In [7]:
from featureextractionsom.functions.data_preparation import featurise_categories

data_raw['VisitorType'] = featurise_categories(data_raw['VisitorType'])

data_raw.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,2,1,1,1,1,1,False,False
1,0,0.0,0,0.0,2,64.000000,0.00,0.10,0.0,0.0,2,2,2,1,2,1,False,False
2,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,2,4,1,9,3,1,False,False
3,0,0.0,0,0.0,2,2.666667,0.05,0.14,0.0,0.0,2,3,2,2,4,1,False,False
4,0,0.0,0,0.0,10,627.500000,0.02,0.05,0.0,0.0,2,3,3,1,4,1,True,False


### 1.4.3 Boolean valus

Replace bool values with 1/0

In [8]:
bool_convert = {True: 1,
                False: 0}

numerical_data = data_raw.replace(bool_convert)
numerical_data.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,2,1,1,1,1,1,0,0
1,0,0.0,0,0.0,2,64.000000,0.00,0.10,0.0,0.0,2,2,2,1,2,1,0,0
2,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,2,4,1,9,3,1,0,0
3,0,0.0,0,0.0,2,2.666667,0.05,0.14,0.0,0.0,2,3,2,2,4,1,0,0
4,0,0.0,0,0.0,10,627.500000,0.02,0.05,0.0,0.0,2,3,3,1,4,1,1,0


In [9]:
numerical_data.dtypes

Administrative               int64
Administrative_Duration    float64
Informational                int64
Informational_Duration     float64
ProductRelated               int64
ProductRelated_Duration    float64
BounceRates                float64
ExitRates                  float64
PageValues                 float64
SpecialDay                 float64
Month                        int64
OperatingSystems             int64
Browser                      int64
Region                       int64
TrafficType                  int64
VisitorType                  int64
Weekend                      int64
Revenue                      int64
dtype: object

## 1.5 Normalise the data

In [10]:
from featureextractionsom.functions.data_preparation import normalise
normalised_df = numerical_data.copy()

for col in numerical_data.columns:
    normalised_df[col]= normalise(numerical_data[col])
    
normalised_df.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,0.0,0.0,0.0,0.0,0.001418,0.000000,1.00,1.00,0.0,0.0,0.0,0.000000,0.000000,0.000,0.000000,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.002837,0.001000,0.00,0.50,0.0,0.0,0.0,0.142857,0.083333,0.000,0.052632,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.001418,0.000000,1.00,1.00,0.0,0.0,0.0,0.428571,0.000000,1.000,0.105263,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.002837,0.000042,0.25,0.70,0.0,0.0,0.0,0.285714,0.083333,0.125,0.157895,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.014184,0.009809,0.10,0.25,0.0,0.0,0.0,0.285714,0.166667,0.000,0.157895,0.0,1.0,0.0


## 1.6 Split data into training and testing sets

Three datasets are required:

- Training data - for training the node matrix
- Validation data - for hyper-parameter tuning
- Testing data - for observing results

In [11]:
from sklearn.model_selection import train_test_split

training_data, test_and_validation_data = train_test_split(normalised_df, 
                                               random_state=1, 
                                               test_size=0.2, 
                                               stratify=normalised_df['Revenue'])
len(training_data)

9864

In [12]:
len(test_and_validation_data)

2466

In [13]:
testing_data, validation_data = train_test_split(test_and_validation_data, 
                                                 random_state=1, 
                                                 stratify=test_and_validation_data['Revenue'], 
                                                 test_size=0.01)
print(len(testing_data))
print(len(validation_data))

2441
25


In [14]:
# Convert data to numPy arrays
training_data = training_data.to_numpy()
testing_data = testing_data.to_numpy()
test_vectors = validation_data.to_numpy()


## 1.7 Store the datasets

Save the training, validation, and testing datasets for use in the next notebook.

In [15]:
from pickle import dump
data_path = 'data'

dump(training_data, open(data_path + '/training_data.pkl', 'wb'))
dump(testing_data, open(data_path + '/testing_data.pkl', 'wb'))
dump(test_vectors, open(data_path + '/test_vectors.pkl', 'wb'))

In [16]:
# Also save the column names for use in notebook 4.
dump(data_raw.columns, open(data_path + '/colnames.pkl', 'wb'))

## References

Sakar, Polat, S.O., Katircioglu, M. et al. (2018) 'Real-time prediction of online shoppers’ purchasing intention using multilayer perceptron and LSTM recurrent neural networks', _Neural Computing and Applications volume_ (31) 6893–6908, available online at [https://link.springer.com/article/10.1007%2Fs00521-018-3523-0](https://link.springer.com/article/10.1007%2Fs00521-018-3523-0) [Accessed 01/03/2020]

Dua, D. and Graff, C. (2019). UCI Machine Learning Repository [http://archive.ics.uci.edu/ml](http://archive.ics.uci.edu/ml). Irvine, CA: University of California, School of Information and Computer Science. [Accessed 01/03/2020]

Matous, F. (2015) '3 ways to get more out of your web analytics', _Harvard Business Review_ [online] available at [https://hbr.org/2015/10/3-ways-to-get-more-out-of-your-web-analytics](https://hbr.org/2015/10/3-ways-to-get-more-out-of-your-web-analytics) [Accessed 05/03/2020]

Builtwith.com (2020) 'Google Analytics Usage Statistics' [online] available at [https://trends.builtwith.com/analytics/Google-Analytics](https://trends.builtwith.com/analytics/Google-Analytics)  [Accessed 13/04/2020]

Fitzpatrick, L. (2019) 'How data analytics impacts small businesses in 2019', _Business.com_ [online] available at [https://www.business.com/articles/the-state-of-data-analytics-in-2019/](https://www.business.com/articles/the-state-of-data-analytics-in-2019/)  [Accessed 05/04/2020]